In [27]:
from openai import OpenAI
from pydantic import BaseModel
from typing import Optional
import json
import inspect

client=OpenAI()

system_message = (
    "You are a customer support agent for ACME Inc."
    "Always answer in a sentence or less."
    "Follow the following routine with the user:"
    "1. First, ask probing questions and understand the user's problem deeper.\n"
    " - unless the user has already provided a reason.\n"
    "2. Propose a fix (make one up).\n"
    "3. ONLY if not satesfied, offer a refund.\n"
    "4. If accepted, search for the ID and then execute refund."
    ""
)

model_id='gpt-3.5-turbo'

def look_up_item(search_query):
    """Use to find item ID.
    Search query can be a description or keywords."""

    # return hard-coded item ID - in reality would be a lookup
    return "item_132612938"


def execute_refund(item_id, reason="not provided"):
    print("Summary:", item_id, reason) # lazy summary
    return "success"


"""define Handoff"""
class Agent(BaseModel):
  name: str='Agent'
  model: str='gpt-3.5-turbo'
  instruction: str='You are a helpful assistant'
  tools: list=[]

class Response(BaseModel):
   agent: Optional[Agent] # current agent
   messages: list

In [28]:

'''test system prompt as a state machine, or condition dispatcher'''
# resp=client.chat.completions.create(
#   model=model_id,
#   messages=[
#     {"role": "system", "content": system_message},
#     {'role': 'user', 'content': 'i have a broken iphone, can i replace it?'},
#     {'role': 'assistant', 'content': 'Do you have AppleCare or device insurance?'},
#     {'role': 'user', 'content': 'no, i don\'t'}
#   ]
# )
# print(resp.choices[0].message)

def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }

def exec_tool_call(tool_call, tools_map, agent_name):
    func_name=tool_call.function.name
    args=json.loads(tool_call.function.arguments)

    print(f"Agent: {agent_name}, exec func: {func_name}({args})")

    return tools_map[func_name](**args)

'''test tools'''
tools=[execute_refund, look_up_item]

def run_full_turn(agent: Agent, messages):
    """
    In practice, we'll also want to let the model use the result to produce another response. 
    That response might also contain a tool call, so we can just run this in a loop until there are no more tool calls.
    """
    curr_agent=agent
    num_init_messages=len(messages)
    messages=messages.copy()
    
    print(messages)

    while True:
      tool_schemas=[function_to_schema(tool) for tool in curr_agent.tools]

      """add tools map, when llm return a FunctionCall msg, call local function based on tools map"""
      tools_map={tool.__name__: tool for tool in curr_agent.tools}

      response = client.chat.completions.create(
          model=agent.model,
          messages=[{"role": "system", "content": curr_agent.instruction}] + messages,
          tools=tool_schemas or None
      )
      message = response.choices[0].message
      messages.append(message)

      '''normal message'''
      if message.content: 
          print(f"{curr_agent.name}:", message.content)

      if not message.tool_calls:
          """when no tool calls, break"""
          break

      """handle function call"""
      # if message.tool_calls[0].function:
      #     print("Func call:", message.tool_calls[0].function)
      for tool_call in message.tool_calls:
          result=exec_tool_call(tool_call, tools_map, curr_agent.name)  # local func call result

          """when response is Agent type, means handoff to other agent"""
          if type(result) is Agent:
              curr_agent=result
              '''rewrite message'''
              result=f"Transfer to Agent: {curr_agent.name}."
              print('Assistant:', result) # print transfer msg
              
              tool_message={
                  'role': 'tool',
                  'tool_call_id': tool_call.id,
                  'content': result
              }
            
              '''when transfer to new agent, previous msg need call completion again?'''
              # print('cur messages:', messages)
              user_last_query=messages[-2]['content']
              forward_user_msg={'role': 'user', 'content': user_last_query}
              
              messages.extend([tool_message, forward_user_msg])
              
          else:
            print('Tool call result:', result)

            """return tool message"""
            tool_message={
                'role': 'tool',
                'tool_call_id': tool_call.id,
                'content': result
            }
            messages.append(tool_message)

    '''return new messages'''
    # return messages[num_init_messages:]
    return Response(
        agent=curr_agent,
        messages=messages[num_init_messages:]
    )


def run_client(messages: list = []):
  # messages = []
  while True:
      user = input("User: ")
      messages.append({"role": "user", "content": user})

      new_messages = run_full_turn(system_message, tools, messages)
      messages.extend(new_messages)

def run_agents():
    agent_refund=Agent(
        name='refund agent',
        instruction='You are a refund agent, help the user with refunds.',
        tools=[execute_refund]
    )

    def place_order(item_name):
        '''mock'''
        return 'order success'

    agent_sales=Agent(
        name='sales agent',
        instruction='You are a sales agent, sell the user a product.',
        tools=[place_order]
    )

    '''mock run turns'''
    messages=[]
    query='place an order for a black boot.'
    print('User:', query)
    messages.append({'role': 'user', 'content': query})

    resp=run_full_turn(agent_sales, messages)
    messages.extend(resp)

    query='Actually, i want a refund'
    print('User: ', query)
    messages.append({'role': 'user', 'content': query})
    resp=run_full_turn(agent_refund, messages)  # manually handoff, not good
    messages.extend(resp)

    print('Final messages: ', messages)



def pretty_print(obj):
    print(json.dumps(obj, indent=2))

'''single turn test'''
# run_full_turn(system_message, [{'role': 'user', 'content': 'Look up the black boot.'}])

'''full turn test'''
# run_client()

'''run agents'''
# run_agents()





'run agents'

In [29]:
def run_agents_with_handoff():
  def escalate_to_human(summary):
    """Only call this if explicitly asked to."""
    print("Escalating to human agent...")
    print("\n=== Escalation Report ===")
    print(f"Summary: {summary}")
    print("=========================\n")
    exit()


  def go_to_sales_agent():
      """User for anything sales or buying related."""
      return sales_agent


  def go_to_issues_and_repairs():
      """User for issues, repairs, or refunds."""
      return issues_and_repairs_agent


  def fallback():
      """Call this if the user brings up a topic outside of your purview,
      including escalating to human."""
      return triage_agent


  triage_agent = Agent(
      name="Triage Agent",
      instructions=(
          "You are a customer service bot for ACME Inc. "
          "Introduce yourself. Always be very brief. "
          "Gather information to direct the customer to the right department. "
          "But make your questions subtle and natural."
      ),
      tools=[go_to_sales_agent, go_to_issues_and_repairs, escalate_to_human],
  )


  def execute_order(product, price: int):
      """Price should be in USD."""
      print("\n\n=== Order Summary ===")
      print(f"Product: {product}")
      print(f"Price: ${price}")
      print("=================\n")
      confirm = input("Confirm order? y/n: ").strip().lower()
      if confirm == "y":
          print("Order execution successful!")
          return "Success"
      else:
          print("Order cancelled!")
          return "User cancelled order."


  sales_agent = Agent(
      name="Sales Agent",
      instructions=(
          "You are a sales agent for ACME Inc."
          "Always answer in a sentence or less."
          "Follow the following routine with the user:"
          "1. Ask them about any problems in their life related to catching roadrunners.\n"
          "2. Casually mention one of ACME's crazy made-up products can help.\n"
          " - Don't mention price.\n"
          "3. Once the user is bought in, drop a ridiculous price.\n"
          "4. Only after everything, and if the user says yes, "
          "tell them a crazy caveat and execute their order.\n"
          ""
      ),
      tools=[execute_order, fallback],
  )


  def look_up_item(search_query):
      """Use to find item ID.
      Search query can be a description or keywords."""
      item_id = "item_132612938"
      print("Found item:", item_id)
      return item_id


  def execute_refund(item_id, reason="not provided"):
      print("\n\n=== Refund Summary ===")
      print(f"Item ID: {item_id}")
      print(f"Reason: {reason}")
      print("=================\n")
      print("Refund execution successful!")
      return "success"


  issues_and_repairs_agent = Agent(
      name="Issues and Repairs Agent",
      instructions=(
          "You are a customer support agent for ACME Inc."
          "Always answer in a sentence or less."
          "Follow the following routine with the user:"
          "1. First, ask probing questions and understand the user's problem deeper.\n"
          " - unless the user has already provided a reason.\n"
          "2. Propose a fix (make one up).\n"
          "3. ONLY if not satesfied, offer a refund.\n"
          "4. If accepted, search for the ID and then execute refund."
          ""
      ),
      tools=[execute_refund, look_up_item, fallback],
  )

  agent = triage_agent
  messages = []
  
  while True:
    query = input("User: ")
    if query == 'bye':
        break
    
    print('User:', query)
    
    messages.append({"role": "user", "content": query})

    response = run_full_turn(agent, messages)
    agent = response.agent
    # print('[curr agent]:', agent.name)
    messages.extend(response.messages)

run_agents_with_handoff()

User: hi
[{'role': 'user', 'content': 'hi'}]
Triage Agent: Hello! How can I assist you today?
User: i want buy a pencil
[{'role': 'user', 'content': 'hi'}, ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None, refusal=None), {'role': 'user', 'content': 'i want buy a pencil'}]
Agent: Triage Agent, exec func: go_to_sales_agent({})
Assistant: Transfer to Agent: Sales Agent.
Sales Agent: I can help you with that! Could you please provide me with the details of the pencil you're interested in, like the brand, type, or any specific features you're looking for?
User: adidas, 3inch, black
[{'role': 'user', 'content': 'hi'}, ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None, refusal=None), {'role': 'user', 'content': 'i want buy a pencil'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(i